In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pprint

In [188]:
f1 = open('west-ham-arsenal-example.txt','r')
file_contents1 = f1.read()
soup = BeautifulSoup(file_contents1,'lxml')

In [189]:
"""
47%	Ball possession	53%
14	Goal attempts	15
3	Shots on goal	4
6	Shots off goal	5
5	Blocked Shots	6
8	Free kicks	13
8	Corner kicks	6
2	Offsides	3
23	Throw ins	22
2	Goalkeeper saves	3
14	Fouls	8
0	Yellow cards	2
 
"""

'\n47%\tBall possession\t53%\n14\tGoal attempts\t15\n3\tShots on goal\t4\n6\tShots off goal\t5\n5\tBlocked Shots\t6\n8\tFree kicks\t13\n8\tCorner kicks\t6\n2\tOffsides\t3\n23\tThrow ins\t22\n2\tGoalkeeper saves\t3\n14\tFouls\t8\n0\tYellow cards\t2\n \n'

In [190]:
home_team = soup.find('td',{'class':'tname-home logo-enable'}).text.strip()
away_team = soup.find('td',{'class':'tname-away logo-enable'}).text.strip()

score_div = soup.find('div',{'id':'content-all'})
scores = score_div.find_all('span',{'class':'scoreboard'})
home_goals = int(scores[0].text.strip())
away_goals = int(scores[1].text.strip())

if home_goals > away_goals:
    result = 'W'
elif home_goals == away_goals:
    result = 'D'
else: result = 'L'

print home_team, away_team
print home_goals, away_goals
print result

West Ham Arsenal
1 5
L


In [191]:
game_date = soup.find('td',{'id':'utime'}).text.strip()[:10]
print game_date

03.12.2016


In [192]:
match_info_trs = soup.find_all('tr',{'class':'content'})

referee = None
attendance = None
venue = None

for match_info in match_info_trs:
    
    
    cur = match_info.text.strip()
    if ('Referee' in cur):
        ref_string = cur.split('Referee: ')
        referee = ref_string[1]
    if ('Attendance' in cur):
        attendance_string = cur.split('Attendance: ')
        attendance = attendance_string[1].strip()
        attendance = attendance[:attendance.index(',')]
        attendance = attendance.replace(' ','')
    if ('Venue' in cur):
        venue_string = cur.split('Venue: ')
        venue = venue_string[1].strip()
    
print "referee:", referee
print "attendance:", attendance
print "venue:", venue

referee: Pawson C. (Eng)
attendance: 56980
venue: Olympic Stadium (London)


In [193]:
whole_game_stats = soup.find('div',{'id':'tab-statistics-0-statistic'})
whole_game_stats = whole_game_stats.findAll('div')

#first_half_stats = soup.find('div',{'id':'tab-statistics-1-statistic'})
#second_half_stats = soup.find('div',{'id':'tab-statistics-2-statistic'})

stats = []
for el in whole_game_stats:
    text = el.text
    if (len(text) > 0):
        stats.append(text)

print stats

home_ball_possession = float(stats[0].strip('%')) / 100
away_ball_possession = float(stats[1].strip('%')) / 100
assert(home_ball_possession + away_ball_possession == 1.0)

home_goal_attempts = int(stats[2])
away_goal_attempts = int(stats[3])

home_shots_on_goal = int(stats[4])
away_shots_on_goal = int(stats[5])

home_shots_off_goal = int(stats[6])
away_shots_off_goal = int(stats[7])

home_blocked_shots = int(stats[8])
away_blocked_shots = int(stats[9])

home_free_kicks = int(stats[10])
away_free_kicks = int(stats[11])

home_corner_kicks = int(stats[12])
away_corner_kicks = int(stats[13])


home_offsides  = int(stats[14])
away_offsides  = int(stats[15])

home_goalkeeper_saves  = int(stats[16])
away_goalkeeper_saves  = int(stats[17])

home_fouls  = int(stats[18]) 
away_fouls  = int(stats[19]) 

home_yellow_cards  = int(stats[20])
away_yellow_cards  = int(stats[21])

[u'50%', u'50%', u'10', u'19', u'3', u'10', u'5', u'6', u'2', u'3', u'13', u'10', u'3', u'2', u'0', u'2', u'5', u'2', u'10', u'11', u'3', u'3']


In [194]:
game_lineups = soup.find('div',{'id':'lineups-content'})
game_lineups = game_lineups.findAll('td')

#remove non_ascii and digits
def remove_badchars(text):
    return ''.join([i if (ord(i) < 128 and 
                          ((ord(i) < ord('0')) or (ord(i) > ord('9')))) 
                    else '' for i in text])



home_formation = game_lineups[0].text
away_formation = game_lineups[2].text

home_players = []
away_players = []

for i in range(4,4+22):
    player_name = remove_badchars(game_lineups[i].text)
    if '(' in player_name:
        player_name = player_name[:player_name.index('(')]
    if (i % 2 == 0):
        home_players.append(player_name)
    else:
        away_players.append(player_name)
        
i += 2
home_substitutes = []
away_substitutes = []

# substitutions made will be a list of pairs (a:b) means a came off, and b came on.
substitutions_made = []

while (game_lineups[i].text != "Missing Players"):
    sub = remove_badchars(game_lineups[i].text)
    sub = sub.replace('(G)','')
    sub = sub.replace('(C)','')
    
    actual_player = sub
    if '(' in sub:
        actual_player = sub[:sub.index('(')]
        substitutions_made.append((actual_player,sub[sub.index('(')+1:-1]))
        
    if (i % 2 != 0):
        home_substitutes.append(actual_player)
    else:
        away_substitutes.append(actual_player)
    i+= 1




        
print substitutions_made
        
print home_substitutes
print away_substitutes
    
# There is no need to collect info on missing players. Including our model the players that are present is enough.
while (game_lineups[i].text != 'Coaches'):
    i += 1

assert(game_lineups[i].text == 'Coaches')
i+= 1

home_coach = remove_badchars(game_lineups[i].text)
i+=1

away_coach = remove_badchars(game_lineups[i].text)

[(u'Arbeloa A.', u'Collins J.'), (u'Elneny M.', u'Coquelin F.'), (u'Ayew A.', u'Fernandes E.'), (u'Carroll A.', u'Noble M.'), (u'Iwobi A.', u'Ozil M.'), (u'Ramsey A.', u'Walcott T.')]
[u'Arbeloa A.', u'Ayew A.', u'Carroll A.', u'Feghouli S.', u'Nordtveit H.', u'Adrian ', u'Zaza S.']
[u'Elneny M.', u'Gibbs K.', u'Holding R.', u'Iwobi A.', u'Ospina D. ', u'Perez L.', u'Ramsey A.']


In [195]:
f2 = open('west-ham-arsenal-players-example.txt','r')
file_contents2 = f2.read()
soup = BeautifulSoup(file_contents2,'lxml')

In [217]:
player_table = soup.find('div',{'id':'tab-player-statistics-0-statistic'})

all_players = []
odd_players = player_table.find_all('tr',{'class':'odd'})
even_players = player_table.find_all('tr',{'class':'even'})



# this info is obtained from each player's <td> in the html
player_fields = ['name','team','goals','assists','goal_attempts','shots_on_goal','blocked_shots','offsides',
             'fouls_committed','fouls_sufferred','yellow_cards','red_cards','pass_success','total_passes']

# these are ALL of the fields that we are writing to the csv, which include player_fields but also include the match_date, and the two team names
all_fields = ['home_team','away_team','game_date'] + player_fields
for player in odd_players + even_players:
    player_tds = player.find_all('td')
    player_info = dict()
    
    player_info['home_team'] = home_team
    player_info['away_team'] = away_team
    player_info['game_date'] = game_date
    
    assert(len(player_tds) == 14)  
    for i in range(0,len(player_tds)):
        player_info[player_fields[i]] = '0' if player_tds[i].text == '-' else player_tds[i].text
    
    
    all_players.append(player_info)



# for player in all_players:
#     for key in player:
#         if (len(player[key]) > 5):
#             print '{0: <12}'.format(player[key]),
#         else:
#             print '{0: <5}'.format(player[key]),
#     print

# for k in range(0,3):
#     for key in all_players[k]:
#         print key, ":",all_players[k][key]
#     print

yellow_cards : 0
shots_on_goal : 0
home_team : West Ham
away_team : Arsenal
name : Adrian
fouls_committed : 0
blocked_shots : 0
pass_success : 0
fouls_sufferred : 0
goal_attempts : 0
assists : 0
goals : 0
team : WES
game_date : 03.12.2016
total_passes : 0
red_cards : 0
offsides : 0

yellow_cards : 0
shots_on_goal : 0
home_team : West Ham
away_team : Arsenal
name : Ayew A.
fouls_committed : 1
blocked_shots : 0
pass_success : 72.7
fouls_sufferred : 3
goal_attempts : 2
assists : 0
goals : 0
team : WES
game_date : 03.12.2016
total_passes : 11
red_cards : 0
offsides : 0

yellow_cards : 0
shots_on_goal : 0
home_team : West Ham
away_team : Arsenal
name : Cech P.
fouls_committed : 0
blocked_shots : 0
pass_success : 35
fouls_sufferred : 0
goal_attempts : 0
assists : 0
goals : 0
team : ARS
game_date : 03.12.2016
total_passes : 20
red_cards : 0
offsides : 0



In [197]:
vars = locals()
vars['home_goals']

1

In [203]:
fieldnames = [
        #basic match info
        'home_team','away_team','game_date','result',
    
        #match statistics
        'home_goals','away_goals','referee','attendance','venue',
        'home_ball_possession', 'away_ball_possession', 'home_goal_attempts', 'away_goal_attempts',
        'home_shots_on_goal', 'away_shots_on_goal', 'home_shots_off_goal', 'away_shots_off_goal',
        'home_blocked_shots', 'away_blocked_shots', 'home_free_kicks', 'away_free_kicks',
        'home_corner_kicks', 'away_corner_kicks', 'home_offsides', 'away_offsides',
        'home_goalkeeper_saves', 'away_goalkeeper_saves', 'home_fouls', 'away_fouls',
        'home_yellow_cards', 'away_yellow_cards',
        #lineups
        'home_formation', 'away_formation', 'home_players', 'away_players', 'home_substitutes', 'away_substitutes', 'substitutions_made',
        'home_coach', 'away_coach',

        #player_info - this will be done in a separate csv
        #'all_players'
                  ]
for field in fieldnames:
    print field, ',',

home_team , away_team , game_date , result , home_goals , away_goals , referee , attendance , venue , home_ball_possession , away_ball_possession , home_goal_attempts , away_goal_attempts , home_shots_on_goal , away_shots_on_goal , home_shots_off_goal , away_shots_off_goal , home_blocked_shots , away_blocked_shots , home_free_kicks , away_free_kicks , home_corner_kicks , away_corner_kicks , home_offsides , away_offsides , home_goalkeeper_saves , away_goalkeeper_saves , home_fouls , away_fouls , home_yellow_cards , away_yellow_cards , home_formation , away_formation , home_players , away_players , home_substitutes , away_substitutes , substitutions_made , home_coach , away_coach ,


In [199]:
import csv
csvfile = open('tertertertertert.csv', 'wb')
csv_writer = csv.writer(csvfile)
csv_writer.writerow(fieldnames)
csv_writer.writerow([home_team,away_team,game_date,result,home_goals , away_goals , referee , attendance , venue , home_ball_possession , away_ball_possession , home_goal_attempts , away_goal_attempts , home_shots_on_goal , away_shots_on_goal , home_shots_off_goal , away_shots_off_goal , home_blocked_shots , away_blocked_shots , home_free_kicks , away_free_kicks , home_corner_kicks , away_corner_kicks , home_offsides , away_offsides , home_goalkeeper_saves , away_goalkeeper_saves , home_fouls , away_fouls , home_yellow_cards , away_yellow_cards , home_formation , away_formation , home_players , away_players , home_substitutes , away_substitutes , substitutions_made, home_coach , away_coach])
csvfile.close()

In [201]:
df = pd.read_csv('tertertertertert.csv')